# **Model 2 : Ktrain**

In [ ]:
# Insatalling ktrain 
# This may be a problem as pip changed some policies so I suggest to use Google Colab
pip install ktrain

In [ ]:
# Importing Genearal Libraries

import pandas as pd
import numpy as np
import sklearn
import re
import string as s
import warnings, gc
warnings.filterwarnings("ignore")

# Tensorflow
import tensorflow as tf

# ktrain
import ktrain
from ktrain import text

# sklearn
from sklearn.model_selection import train_test_split


In [ ]:
# importing data
# change read method according to dataset
train_sm_df = pd.read_json("/content/drive/My Drive/train_extra.json")
test_df=pd.read_json("/content/drive/My Drive/embold_test.json")

In [ ]:
# splitting data
target = ['label'] 
data = ['text']
X = train_sm_df[data]
y = train_sm_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

In [ ]:
# Transformer Model
model_ = 'bert-base-uncased'
t_mod = text.Transformer(model_, maxlen=100, classes = [0,1,2])


'''Converting split data to list [so it can processed]'''
#train
X_tr = X_train['text'].tolist()
y_tr = y_train['label'].tolist()

#test
X_ts = X_test['text'].tolist()
y_ts = y_test['label'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

# increasing batch size helps in increasing speed of gpu as we can do parallel computation 
# increasing too much will also lead to poor results as it would be taking average of all the local optimal points
# It may happen that local and optimal solution have high differences which will effect the avg value
# decreasing batch size too much may also give bad results : because it may lead to convergence at local optimal minima 
# you have to tune this accordng to need
learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=8) # preferable batch size = 8,16,

In [ ]:
# Model Train
# as an output you will get summary will
learner.fit_onecycle(learning_rate, epochs)

In [ ]:
# Prediction
# pass The name of classes as you want 
classes = ['1', '2','3']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)
# pass the array or string as you like to test your data
pred_class = predictor.predict(X_test['text'][67])
print("Predicted Class: ", classes[pred_class])